<a href="https://colab.research.google.com/github/Ryan-Riggs/myrepo/blob/master/Landsat_widths_flags.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import ee
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=3h71Nn5x4JMfsCXh5egqdshpUlthPVbbDo6BxdTZIag&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/3QGa86nZgk8DZQPcEM3SDhPWiBWKAaYL63qRNmNAY6r_AWB31VYawaw

Successfully saved authorization token.


In [ ]:
x = 'users/eeProject/RivWidthCloudPaper:functions_Landsat578/functions_waterClassification_Jones2019.py'
fls = 'users/eeProject/RivWidthCloudPaper:functions_Landsat578/functions_landsat.py'
flsh = 'users/eeProject/RivWidthCloudPaper:rwc_landsat.py'
fnsLandsat = 'users/eeProject/RivWidthCloudPaper:functions_Landsat578/functions_landsat.py'
lsFun = 'users/eeProject/RivWidthCloudPaper:functions_Landsat578/functions_landsat.py'
riverFun = 'users/eeProject/RivWidthCloudPaper:functions_river.py'
grwl_cline = ee.FeatureCollection('users/eeProject/GRWL_summaryStats')
#fc_3spc = ee.FeatureCollection("users/rriggs/east_validation_1spc_3x/eastern_merged")
fc_3spc = ee.FeatureCollection("users/rriggs/na_sj_using_R_min_100")
gauges = ee.FeatureCollection("users/rriggs/Gauge_points")


In [ ]:
jrcSummary = ee.Image("JRC/GSW1_0/GlobalSurfaceWater")
occ = jrcSummary.select('occurrence')
jrcMeta = ee.Image("JRC/GSW1_0/Metadata")
nobs = jrcMeta.select('valid_obs')
change = jrcSummary.select('change_abs')
abs_change = change.abs()
waterMax = occ.gte(0)
grwl = ee.FeatureCollection("users/eeProject/grwl")
# create image with bands ranging from quantile 0-100
quantileBreaks = ee.List.sequence(0, 100, 1)
import folium
def creatQuantileImage(l, prev):
    return(ee.Image(prev).addBands(occ.gte(ee.Image.constant(l)).rename([ee.String('q').cat(ee.Number(l).format('%03d'))])))


# rename the flags so that their value won't be replaced in the next step
# when calculating the mean river mask values
def appendStringFlag(l):
    return(ee.String(l).cat('_flag'))



In [ ]:
riverMask = ExtractChannel(waterMax)
quantileImage = ee.Image(quantileBreaks.iterate(creatQuantileImage, ee.Image())).select('^q.*').updateMask(riverMask).unmask(0)
bn = ee.List(quantileImage.bandNames()).map(appendStringFlag)
reducer = ee.Reducer.anyNonZero().forEach(bn)

In [ ]:
def Mndwi(image):
    return(image.normalizedDifference(['Green', 'Swir1']).rename('mndwi'))

def Mbsrv(image):
    return(image.select(['Green']).add(image.select(['Red'])).rename('mbsrv'))

def Mbsrn(image):
    return(image.select(['Nir']).add(image.select(['Swir1'])).rename('mbsrn'))

def Ndvi(image):
    return(image.normalizedDifference(['Nir', 'Red']).rename('ndvi'))

def Awesh(image):
    return(image.expression('Blue + 2.5 * Green + (-1.5) * mbsrn + (-0.25) * Swir2', {
    'Blue': image.select(['Blue']),
    'Green': image.select(['Green']),
    'mbsrn': Mbsrn(image).select(['mbsrn']),
    'Swir2': image.select(['Swir2'])}))

def Dswe(i):
    mndwi = Mndwi(i)
    mbsrv = Mbsrv(i)
    mbsrn = Mbsrn(i)
    awesh = Awesh(i)
    swir1 = i.select(['Swir1'])
    nir = i.select(['Nir'])
    ndvi = Ndvi(i)
    blue = i.select(['Blue'])
    swir2 = i.select(['Swir2'])

    t1 = mndwi.gt(0.124)
    t2 = mbsrv.gt(mbsrn)
    t3 = awesh.gt(0)
    t4 = (mndwi.gt(-0.44)
    .And(swir1.lt(900))
    .And(nir.lt(1500))
    .And(ndvi.lt(0.7)))
    t5 = (mndwi.gt(-0.5)
    .And(blue.lt(1000))
    .And(swir1.lt(3000))
    .And(swir2.lt(1000))
    .And(nir.lt(2500)))

    t = t1.add(t2.multiply(10)).add(t3.multiply(100)).add(t4.multiply(1000)).add(t5.multiply(10000))

    noWater = (t.eq(0)
    .Or(t.eq(1))
    .Or(t.eq(10))
    .Or(t.eq(100))
    .Or(t.eq(1000)))
    hWater = (t.eq(1111)
    .Or(t.eq(10111))
    .Or(t.eq(11011))
    .Or(t.eq(11101))
    .Or(t.eq(11110))
    .Or(t.eq(11111)))
    mWater = (t.eq(111)
    .Or(t.eq(1011))
    .Or(t.eq(1101))
    .Or(t.eq(1110))
    .Or(t.eq(10011))
    .Or(t.eq(10101))
    .Or(t.eq(10110))
    .Or(t.eq(11001))
    .Or(t.eq(11010))
    .Or(t.eq(11100)))
    pWetland = t.eq(11000)
    lWater = (t.eq(11)
    .Or(t.eq(101))
    .Or(t.eq(110))
    .Or(t.eq(1001))
    .Or(t.eq(1010))
    .Or(t.eq(1100))
    .Or(t.eq(10000))
    .Or(t.eq(10001))
    .Or(t.eq(10010))
    .Or(t.eq(10100)))

    iDswe = (noWater.multiply(0)
    .add(hWater.multiply(1))
    .add(mWater.multiply(2))
    .add(pWetland.multiply(3))
    .add(lWater.multiply(4)))

    return(iDswe.rename(['dswe']))

def ClassifyWaterJones2019(img):
    dswe = Dswe(img)
    waterMask = dswe.eq(1).Or(dswe.eq(2)).rename(['waterMask'])
    return(waterMask)

def switchGeometryLine2Endpoints(f):
        f = f.set({'lineGeometry': f.geometry()})
        l = f.geometry().coordinates()
        g = ee.Geometry.MultiPoint(l, 'EPSG:4326')
        return(f.setGeometry(g))

def switchGeometryEndpoints2Line(f):
        return(f.setGeometry(f.get('lineGeometry')).set('lineGeometry', None))

def GetCenterline(clDataset, bound):
    # // filter the GRWL centerline based on area of interest
    cl = clDataset.filterBounds(bound)
    return(cl)

def ExtractChannel(image):
    # // extract the channel water bodies from the water mask, based on connectivity to the reference centerline.
    connectedToCl = (image.Not().cumulativeCost(
        source = ee.Image().toByte().paint(grwl_cline, 1).And(image), #// only use the centerline that overlaps with the water mask
        maxDistance = 4000,
        geodeticDistance = False).eq(0))

    channel = image.updateMask(connectedToCl).unmask(0).updateMask(image.gte(0)).rename(['channelMask'])
    return(channel)


def RemoveIsland(channel):
    # /* fill in island as water if the size (number of pixels) of the island is smaller than FILL_SIZE */
    fill = channel.Not().selfMask().connectedPixelCount(333).lt(333)
    river = channel.where(fill, ee.Image(1)).rename(['riverMask'])
    return(river)

def ExtractRiver(imgIn, clData, maxDist, minIslandRemoval):
    waterMask = imgIn.select(['waterMask'])
    bound = waterMask.geometry()
    cl = GetCenterline(clData, bound)
    channelMask = ExtractChannel(waterMask, cl, maxDist)
    riverMask = RemoveIsland(channelMask, minIslandRemoval)
    return(imgIn.addBands(channelMask).addBands(riverMask))

def widths(image):
  width = (image.eq(1).reduceRegions(
  collection = filt, 
  reducer= ee.Reducer.mean(),
  ))
  flags = (image.reduceRegions(
  collection= width.map(switchGeometryLine2Endpoints), 
  reducer= ee.Reducer.max(),
))
  return(flags)

def filt_lines (f):
  return f.set('geo_type', f.geometry().type())

def maximum_no_of_tasks(MaxNActive, waitingPeriod):
	"""maintain a maximum number of active tasks
	"""
	import time
	import ee
	ee.Initialize()

	time.sleep(10)
	## initialize submitting jobs
	ts = list(ee.batch.Task.list())

	NActive = 0
	for task in ts:
		if ('RUNNING' in str(task) or 'READY' in str(task)):
			NActive += 1
	## wait if the number of current active tasks reach the maximum number
	## defined in MaxNActive
	while (NActive >= MaxNActive):
		time.sleep(waitingPeriod) # if reach or over maximum no. of active tasks, wait for 2min and check again
		ts = list(ee.batch.Task.list())
		NActive = 0
		for task in ts:
			if ('RUNNING' in str(task) or 'READY' in str(task)):
				NActive += 1
	return()
def buffer_zone (f):
  return f.buffer(5000)
def  ftr_coll (f):
  return ee.FeatureCollection(f)

def distance_fun (f):
  l = f.geometry().distance(filter_gauge.geometry())
  d = f.set('distance', l)
  return(d)

In [ ]:
path = [10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33]
row = [27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41]

p1 = [9427520, 9429010, 9429100, 9429300, 9429500, 9522200, 9424000, 9429490, 9520700, 9404200, 9520360, 8329918, 8329928, 8330150, 8332000, 8332010, 
     11383800, 11425000, 11425500, 11447500, 9163500, 9180500, 9315000, 9379910, 9380000, 9380002, 9383000, 7117500, 9357000, 11378000, 11477000, 14322900, 
     11530500, 13090355, 13134500, 13135000, 13172500, 13173600, 13213100, 9272400, 13057150, 13057155, 13062504, 13077000, 13081500, 13082000, 9307000, 6646800, 
     6652000, 6674500, 6759910, 6644500, 6645500, 6650000, 6684500, 6760500 
]

p = [7355500,
7374000,
7381490,
8041000,
8041780,
8066000,
8066250,
8067000,
8114000,
8114500,
8116650,
8117290,
2330166,
2343260,
2343500,
2343801,
2344000,
2353000,
2355662,
2356000,
2358000,
2358700,
2358754,
2365200,
2365500,
2366500,
2372422,
2375500,
2428400,
2429500,
2469761,
2470000,
2479000,
2489000,
7374525,
2203536,
2215500,
2224940,
2225000,
2226000,
2228070,
2231253,
2231254,
2232000,
2234500,
2236000,
2236125,
2312600,
2312720,
2323000,
2323500,
2323592,
9424000,
9427520,
9429010,
9429100,
9429300,
9429490,
9429500,
9520700,
9522200,
9404200,
9520360,
8329918,
8329928,
8330150,
8332000,
8332010,
7229000,
7229050,
7229100,
7229200,
7332000,
7159100,
7159400,
7160000,
7161000,
7161450,
7228500,
7245000,
7246500,
7249455,
7250500,
7250550,
7250551,
7258000,
7263450,
7263500,
7265450,
7288800,
7288955,
7289000,
7335500,
7336820,
7337000,
7341500,
7344400,
7348500,
7350500,
7358000,
7359001,
7359002,
7359500,
7364100,
7367000,
7367005,
7367630,
7032000,
7061000,
7074500,
7074850,
7076750,
7077000,
7077800,
7165570,
7165600,
7193500,
7194500,
2335815,
2338500,
2339500,
2341500,
2341505,
2342881,
2349500,
2349605,
2394000,
2397000,
2400500,
2407000,
2408000,
2409500,
2411000,
2414715,
2418000,
2420000,
2423000,
2427500,
2437100,
2437500,
2444160,
2444500,
2447025,
2449500,
2452500,
2462500,
2462951,
2465000,
2466000,
2466030,
2467000,
3518000,
3518300,
3520000,
3535912,
3538150,
3544000,
3555000,
3566000,
3568000,
3571850,
3573500,
3575500,
3593500,
2102500,
2105769,
2121000,
2122500,
2127500,
2129000,
2131000,
2131010,
2135200,
2142500,
2146000,
2147000,
2147020,
2148000,
2153500,
2153551,
21564493,
2156500,
2160750,
2161000,
2161500,
2169000,
2169500,
2171645,
2171650,
2171700,
2172002,
2195000,
2196484,
2197000,
2197320,
21973269,
2197500,
2198375,
2212735,
2224500,
3454500,
3469000,
3497000,
11383800,
11425000,
11425500,
11447500,
9163500,
9180500,
9315000,
9379910,
9380000,
9380002,
9383000,
9357000,
7117500,
6879100,
6887500,
6888345,
6888350,
7144550,
7145200,
7146500,
7150500,
7152500,
7157980,
7158000,
7163500,
7164500,
5585500,
5586100,
5587450,
5587455,
5587500,
6818000,
6889000,
6891000,
6891080,
6892350,
6892500,
6893000,
6895500,
6906500,
6909000,
6922450,
6926000,
6926500,
6926510,
6934500,
6935965,
7010000,
7054000,
7055000,
7057370,
7060500,
7178600,
7190500,
3255000,
3270400,
3270500,
3271601,
3277200,
3284000,
3287000,
3290500,
3294500,
3303280,
3303500,
3316500,
3320000,
3322420,
3340500,
3342000,
3343000,
3343010,
3352953,
3353000,
3353611,
3364000,
3374000,
3374100,
3377500,
3381700,
3384500,
3399800,
3417500,
3418420,
3425000,
3426500,
3431500,
3435000,
3437000,
3609500,
3611500,
7020500,
7022000,
2026000,
2029000,
2035000,
2079500,
3062000,
3072500,
3072655,
3115000,
3150000,
3150300,
3151000,
3159530,
3159870,
3160000,
3163000,
3164500,
3168000,
3171000,
3176500,
3184500,
3189500,
3193000,
3198000,
3201500,
3206000,
3216000,
3216600,
3227200,
3227500,
3238000,
3490350,
3491500,
1474500,
1577000,
1578310,
1618000,
1636500,
1638500,
1646500,
1646502,
1668000,
2037500,
2080500,
11378000,
11477000,
11530500,
14322900,
13090355,
13134500,
13135000,
13172500,
13173600,
13213100,
9272400,
9307000,
13057150,
13057155,
13062504,
13077000,
13081500,
13082000,
6644500,
6645500,
6646800,
6650000,
6652000,
6674500,
6684500,
6686000,
6686500,
6759910,
6760500,
6764000,
6452000,
6463720,
6465000,
6465500,
6467500,
6478500,
6486000,
6601200,
6768000,
6770000,
6770200,
6770500,
6774000,
6778000,
6780000,
6785000,
6788500,
6790500,
6790501,
6793000,
6794500,
6796000,
6801000,
6805500,
5383500,
5407000,
5411500,
5416100,
5420460,
5420500,
5446500,
5447800,
5455700,
5458300,
5464000,
5464315,
5464420,
5464500,
5465000,
5465500,
5465700,
5474500,
5482000,
5484600,
5485500,
5488110,
5488500,
5489500,
5490500,
5490600,
5570500,
6610000,
6807000,
6813500,
3327500,
3329000,
3329500,
3335500,
3336000,
4073500,
4101500,
4102000,
4108660,
4119000,
4119300,
4182900,
4191500,
4192500,
5404000,
5406000,
5427530,
5427570,
5430500,
5437500,
5440700,
5441500,
5442300,
5443500,
5520500,
5526000,
5527000,
5527500,
5533500,
5533600,
5534050,
5534100,
5536890,
5536995,
5537000,
5537980,
5538000,
5539660,
5541000,
5543500,
5548500,
5551000,
5551540,
5552500,
5553500,
5558300,
5568500,
1542500,
3012600,
3015310,
3016000,
3025500,
3031500,
3033000,
3036500,
3039035,
3044000,
3049500,
3075070,
3083500,
3085000,
3086000,
3107500,
3111534,
3140500,
3144500,
4157000,
4159130,
1144500,
1154500,
1156500,
1172000,
1172003,
1172010,
1184000,
1193050,
1326500,
1327750,
1329650,
1331095,
1335500,
1335754,
1356000,
1357500,
1358000,
1372058,
1427510,
1434000,
1438500,
1440200,
1446500,
1457500,
1462000,
1463500,
1473500,
1503000,
1513831,
1515000,
1515050,
1531500,
1533400,
1536500,
1540500,
1545500,
1551500,
1553500,
1554000,
1567000,
1570500,
1576000,
4223500,
4231600,
4235600,
4247055,
1067000,
1088500,
1092000,
1100000,
12150800,
12453700,
12462600,
12464500,
14103000,
14144700,
14174000,
14191000,
14197900,
14198000,
14207700,
14211720,
14223780,
14246900,
12353000,
12388700,
12389000,
12391000,
12417610,
12436500,
12437900,
12450700,
12472800,
12508990,
12514000,
13269000,
13290000,
13334300,
13342500,
13343000,
13353000,
6041500,
6042000,
6066500,
6071500,
6074000,
6078200,
6090300,
6090800,
6109000,
6115000,
6115200,
6186500,
6214500,
6217500,
6218000,
6294000,
6294500,
6294700,
6295000,
6296120,
6309000,
6327500,
6329500,
6339000,
6342500,
6440000,
4024000,
5242300,
5261000,
5267000,
5270700,
5283500,
5288500,
5331000,
5331580,
5339500,
5340500,
5344500,
5355250,
5360000,
5361000,
5362500,
5365550,
5367000,
5369500,
5378500,
4059000,
4063500,
4065722,
4066000,
4067000,
4067500,
4077400,
4079000,
4084445,
4084475,
4084500,
4085054,
4085059,
4125550,
4126000,
5395000,
5398000,
5400760,
5400800,
1139500,
4264331,
1010000,
1010500,
1012500,
1014000,
1015000,
1017500,
1028000,
1030000,
1034500,
1036000,
1036390,
1042500,
1046500,
1047150,
1048500,
1049205,
1049265,
1059000,
1131500,
12196150,
12199000,
12200500,
12303000,
12305000,
12306000,
12318500,
12322000,
12369000,
12392000,
12395500,
12396500,
12398500,
12399500,
12438000,
6109500,
6132000,
6177000,
6185500,
6330000,
5133500,
15302000,
15302300,
15302500,
15300000,
15300500,
15129000,
15024800,
15041200,
15056500,
15565447,
15303900,
15304000,
15266110,
15266150,
15292700,
15292780,
15294345,
15294350,
15294500,
15303600,
15202000,
15212000,
15214000,
15281000,
15291000,
15291500,
15292000,
15472000,
15746000,
15565200,
15744000,
15744500,
15468000,
15564800,
15564900,
15389100,
15389500,
15453500,
15478000,
15515500,
15518300,
15356000,
15389000,
15875000,
15880000,
8154000,
8038500,
8066191,
23432415,
2359101,
2232400,
2234000,
2234010,
9500500,
8314500,
8359500,
8382800,
7302000,
8045500,
8063000,
8088600,
7244500,
7341301,
7357000,
7357501,
7192500,
2399500,
2402500,
2405000,
2412500,
2416000,
3519500,
3543005,
3585500,
3589500,
2097000,
2187000,
2187252,
2187500,
2189000,
2218500,
2220500,
3507000,
9421500,
9335000,
9339500,
9120500,
6857000,
6886000,
7148140,
7164000,
7164200,
6920500,
6922000,
6922500,
6925000,
7049500,
7053000,
7059000,
7191500,
3426310,
3438220,
2079000,
3071500,
3180000,
2041500,
11366500,
11368500,
11369000,
11369500,
14335000,
13088000,
13171000,
13171620,
13031500,
6453000,
5487500,
3332500,
3332555,
4087010,
4087170,
4102533,
4120250,
5547350,
5560000,
1322500,
4234061,
14105700,
14235000,
12391400,
13343500,
13343600,
14019200,
6058000,
6058502,
6286250,
6338490,
5341550,
5344490,
40590345,
4067651,
4071775,
40851385,
4136500,
5400000,
4137500,
12301500,
12301850,
12405000,
12405500,
15258000,
2359170,
2359230,
2470500,
2492600,
2244040,
2244450,
2246500,
2084472]

collection = fc_3spc.map(filt_lines)
fc_3spc = collection.filter(ee.Filter.eq('geo_type', 'LineString'))

#fc_3spc = fc_3spc.filterBounds(rc_roi)
LT5_BANDS = ['B1',   'B2',    'B3',  'B4',  'B5',    'B7',    'B6'];
LE7_BANDS = ['B1',   'B2',    'B3',  'B4',  'B5',    'B7',    'B6_VCID_1'];
LC8_BANDS = ['B2',   'B3',    'B4',  'B5',  'B6',    'B7',    'B10'];
STD_NAMES = ['Blue', 'Green', 'Red', 'Nir', 'Swir1', 'Swir2', 'Temp'];

#// load Landsat 5,7,8 collections:
#// TODO(GHA): combine 5, 7, and 8 collections:
LT5 = ee.ImageCollection('LANDSAT/LT5_L1T_TOA').select(LT5_BANDS, STD_NAMES);
LT5 = ee.ImageCollection('LANDSAT/LT05/C01/T1_TOA').select(LT5_BANDS, STD_NAMES); 
##LE7 = ee.ImageCollection('LANDSAT/LE07/C01/T1_TOA')
##.select(LE7_BANDS, STD_NAMES);
LC8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA').select(LC8_BANDS, STD_NAMES);

collection = LC8.merge(LT5)
filtered = collection.filterDate('1984-01-01', '2020-12-31').sort('system:time_start').filterMetadata('CLOUD_COVER', 'less_than', 10).filterBounds(fc_3spc)
distinctRows = ee.List(filtered.distinct(['WRS_ROW']).aggregate_array('WRS_ROW'));
distinctPaths = ee.List(filtered.distinct(['WRS_PATH']).aggregate_array('WRS_PATH'));


#filtered = collection.filterDate('2011-01-01', '2020-12-31').sort('system:time_start').filterMetadata('CLOUD_COVER', 'less_than', 10).filterBounds(fc_3spc)#filtered = filtered.sort('system:time_start')

In [ ]:
dp = list(distinctPaths.getInfo()) 

In [ ]:
######################################################################################### calculate median GSW change across xsections. 
for i in range(len(p)):
  filter_gauge = gauges.filter(ee.Filter.eq('SITE_NUM', p[i]))
  buff1 = filter_gauge.map(buffer_zone)
  fi1 = buff1.map(ftr_coll)
  filt1 = fc_3spc.filterBounds(fi1.flatten().geometry())
  fff = filt1.map(distance_fun)
  filt = fff.limit(5, 'distance')
  result = (abs_change
    ##.addBands(nobs.rename(['n_valid_obs']))
      .reduceRegions(
      collection = filt,
      reducer =  ee.Reducer.median(),
      scale = 30,
      crs = 'EPSG:4326',
      tileScale = 1
      ))
  task = (ee.batch.Export.table.toDrive(
  collection = result,
  description = 'widths_' + '_' + str(p[i]),
  folder = 'python_gauge_w_change',
  fileNamePrefix = 'Gauge_change' + '_' + str(p[i]),
  fileFormat = 'csv'
  ))

  task.start()
#print(output.first())
  print('task', i, 'has started')
  maximum_no_of_tasks(3, 60)

In [ ]:
#################### Calculate quantile widths for each gauge/scene. 
for i in range(len(p)):
  filter_gauge = gauges.filter(ee.Filter.eq('SITE_NUM', p[i]))
  buff1 = filter_gauge.map(buffer_zone)
  fi1 = buff1.map(ftr_coll)
  filt1 = fc_3spc.filterBounds(fi1.flatten().geometry())
  fff = filt1.map(distance_fun)
  filt = fff.limit(5, 'distance')
  xsections = filt
  xsectionEndpoints = xsections.map(switchGeometryLine2Endpoints)
  result = (quantileImage
  .reduceRegions(
  collection = xsectionEndpoints,
  reducer = reducer,
  scale = 30,
  crs = 'EPSG:4326',
  tileScale = 1)
  .map(switchGeometryEndpoints2Line))

    # calculate the mean value for the river masks

  result = (quantileImage
  .addBands(nobs.rename(['n_valid_obs']))
  .reduceRegions(
  collection = result,
  reducer =  ee.Reducer.mean(),
  scale = 30,
  crs = 'EPSG:4326',
  tileScale = 1
  ))

  result = (abs_change
  ##.addBands(nobs.rename(['n_valid_obs']))
  .reduceRegions(
  collection = result,
  reducer =  ee.Reducer.median(),
  scale = 30,
  crs = 'EPSG:4326',
  tileScale = 1
  ))


    # export widths
  task = (ee.batch.Export.table.toDrive(
  collection = result,
  description = 'NA' + '_' + str(p[i]),
  folder = 'NA_quantiles',
  fileNamePrefix = 'NA_' + '_' + str(p[i]),
  fileFormat = 'csv'
  ))

  task.start()
#print(output.first())
  print('task', i, 'has started')
  maximum_no_of_tasks(3, 60)

task 0 has started
task 1 has started
task 2 has started
task 3 has started
task 4 has started
task 5 has started
task 6 has started
task 7 has started
task 8 has started
task 9 has started
task 10 has started
task 11 has started
task 12 has started
task 13 has started
task 14 has started
task 15 has started
task 16 has started
task 17 has started
task 18 has started
task 19 has started
task 20 has started
task 21 has started
task 22 has started
task 23 has started
task 24 has started
task 25 has started
task 26 has started
task 27 has started
task 28 has started
task 29 has started
task 30 has started
task 31 has started
task 32 has started
task 33 has started
task 34 has started
task 35 has started
task 36 has started
task 37 has started
task 38 has started
task 39 has started
task 40 has started
task 41 has started
task 42 has started
task 43 has started
task 44 has started
task 45 has started
task 46 has started
task 47 has started
task 48 has started
task 49 has started
task 50 ha

In [ ]:
####### Calculate widths for each gauge in list p
for i in range(len(p)):
  filter_gauge = gauges.filter(ee.Filter.eq('SITE_NUM', p[i]))
  buff1 = filter_gauge.map(buffer_zone)
  fi1 = buff1.map(ftr_coll)
  filt1 = fc_3spc.filterBounds(fi1.flatten().geometry())
  fff = filt1.map(distance_fun)
  filt = fff.limit(5, 'distance')
  #print(filt)
  #filt = fc_3spc.filterBounds(x1)
  filtered = collection.filterDate('1984-01-01', '2020-12-31').sort('system:time_start').filterMetadata('CLOUD_COVER', 'less_than', 10).filterBounds(filt)
  waterMask = filtered.map(ClassifyWaterJones2019)
  GetCenterline(grwl_cline, filt)
  riverMask = waterMask.map(ExtractChannel)
  riverMask_noislands = riverMask.map(RemoveIsland)
  w = riverMask_noislands.map(widths)
  tbl = w.flatten()
  sel_xsections = tbl.filter(ee.Filter.gt('mean', 0)).filter(ee.Filter.eq('max', 0))
  output = sel_xsections.select(
    ['ID', 'mean', 'date', 'width_m', 'lat_dd', 'lon_dd', 'ID_2', 'max','COMID'])
      # export widths
  task = (ee.batch.Export.table.toDrive(
  collection = output,
  description = 'widths_' + '_' + str(p[i]),
  folder = 'Scene_test',
  fileNamePrefix = 'Gauge_' + '_' + str(p[i]),
  fileFormat = 'csv'
  ))

  task.start()
#print(output.first())
  print('task', i, 'has started')
  maximum_no_of_tasks(3, 60)

task 0 has started
task 1 has started
task 2 has started


In [ ]:
def ind (f):
  buff1 = ee.FeatureCollection(f).map(buffer_zone)
  fi1 = buff1.map(ftr_coll)
  filt1 = fc_3spc.filterBounds(fi1.flatten().geometry())
  fff = filt1.map(distance_fun)
  filt = fff.limit(5, 'distance')
  return(filt)

In [ ]:
##Calculate widths for each landsat row/path from 1984-2020

##for i in range(len(distinctPaths)):
filtered = collection.filterDate('1984-01-01', '2020-12-31').sort('system:time_start').filterMetadata('CLOUD_COVER', 'less_than', 10).filterMetadata('WRS_PATH', 'equals', 23).filterMetadata('WRS_ROW', 'equals', 39)
filter_gauge = gauges.filterBounds(filtered.geometry())
#buff1 = filter_gauge.map(buffer_zone)
#fi1 = buff1.map(ftr_coll)
#filt1 = fc_3spc.filterBounds(fi1.flatten().geometry())
#fff = filt1.map(distance_fun)
#filt = fff.limit(5, 'distance')
  #print(filt)
  #filt = fc_3spc.filterBounds(x1)
filt = filter_gauge.map(ind)
filt = filt.flatten()
waterMask = filtered.map(ClassifyWaterJones2019)
GetCenterline(grwl_cline, filt)
riverMask = waterMask.map(ExtractChannel)
riverMask_noislands = riverMask.map(RemoveIsland)
w = riverMask.map(widths)
tbl = w.flatten()
sel_xsections = tbl.filter(ee.Filter.gt('mean', 0)).filter(ee.Filter.eq('max', 0))
output = sel_xsections.select(
 ['ID', 'mean', 'date', 'width_m', 'lat_dd', 'lon_dd', 'ID_2', 'max','COMID'])
      # export widths
task = (ee.batch.Export.table.toDrive(
collection = output,
description = 'widths_',
folder = 'Scene_test',
fileNamePrefix = 'Gauge_',
fileFormat = 'csv'
))

task.start()
#print(output.first())
#print('task', i, 'has started')
print('task has started')
maximum_no_of_tasks(3, 60)


NameError: ignored

In [ ]:
 maximum_no_of_tasks(4, 30)

()